In [ ]:
### HYPEROPT / optional task for the student
# extend WandB.ai integration in the code with sweeps
# (e.g. add variables like learning rate, optimizer, neurons_FC1, neurons_FC2)
# help: https://docs.wandb.ai/guides/sweeps and
#       https://github.com/wandb/wandb/issues/5003
# store the hyperparameters and val_acc to wandb
# run at least 20 trainings
# in wandb.ai, export the result of the runs as a .csv file,
# in wandb.ai, create a report from the sweep results, share by public link,

In [1]:
# install pytorch lithening
!pip install pytorch-lightning --quiet
!pip install wandb -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


In [2]:
import pytorch_lightning as pl
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import CIFAR10
from pytorch_lightning.loggers import WandbLogger
import wandb
import pprint



In [3]:
# create one class to deal with data
class CifarDataModule(pl.LightningDataModule):
  def __init__(self, batch_size, data_dir="./"):
    super().__init__()
    self.data_dir=data_dir
    self.batch_size=batch_size
    self.transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
    self.num_classes=10

  def prepare_data(self):
    CIFAR10(self.data_dir,train=True,download=True)
    CIFAR10(self.data_dir,train=False,download=True)

  def setup(self, stage=None):
    if stage=='fit' or stage is None:
      cifar_full=CIFAR10(self.data_dir,train=True,transform=self.transform)
      self.cifar_train,self.cifar_val=random_split(cifar_full,[45000,5000])

    if stage=='test' or stage is None:
      self.cifar_test=CIFAR10(self.data_dir,train=False,transform=self.transform)

  def train_dataloader(self):
    return DataLoader(self.cifar_train,batch_size=self.batch_size,shuffle=True,num_workers=2)

  def val_dataloader(self):
    return DataLoader(self.cifar_val,batch_size=self.batch_size,shuffle=False,num_workers=2)

  def test_dataloader(self):
    return DataLoader(self.cifar_test,batch_size=self.batch_size,shuffle=False,num_workers=2)




In [4]:
class CIFAR10LitModel(pl.LightningModule):
    def __init__(self, config, input_shape, num_classes):
      super().__init__()
      self.input_shap = input_shape
      self.learning_rate = config.learning_rate
      self.optimizer = config.optimizer
      self.fc1_num = config.neurons_FC1
      self.fc2_num =  config.neurons_FC2

      # model architecture
      self.conv1=nn.Conv2d(3,32,3,1)
      self.conv2=nn.Conv2d(32,32,3,1)
      self.conv3=nn.Conv2d(32,64,3,1)
      self.conv4=nn.Conv2d(64,64,3,1)
      self.pool1=nn.MaxPool2d(2)
      self.pool2=nn.MaxPool2d(2)

      n_sizes = self._get_output_shape(input_shape)
      self.fc1=nn.Linear(n_sizes,self.fc1_num)
      self.fc2=nn.Linear(self.fc1_num,self.fc2_num)
      self.fc3=nn.Linear(self.fc2_num,num_classes)

      self.train_acc=Accuracy(task='multiclass',num_classes=10)
      self.val_acc=Accuracy(task='multiclass',num_classes=10)
      self.test_acc=Accuracy(task='multiclass',num_classes=10)


    def _get_output_shape(self, shape):
          '''returns the size of the output tensor from the conv layers'''
          batch_size = 1
          input = torch.autograd.Variable(torch.rand(batch_size, *shape))
          output_feat = self._feature_extractor(input)
          n_size = output_feat.data.view(batch_size, -1).size(1)
          return n_size


  # conv1,relu, conv2,relu, maxpool,conv3,relu,conv4,relu,maxpool
    def _feature_extractor(self,x):
      x=F.relu(self.conv1(x))
      x=self.pool1(F.relu(self.conv2(x)))
      x=F.relu(self.conv3(x))
      x=self.pool2(F.relu(self.conv4(x)))
      return x


    def forward(self,x):
      x=self._feature_extractor(x)
      x=x.view(x.size(0),-1)
      x=F.relu(self.fc1(x))
      x=F.relu(self.fc2(x))
      x=F.log_softmax(self.fc3(x),dim=1)
      return x

    def training_step(self, batch, batch_idx):
      x, y = batch
      logits = self(x)
      loss = F.nll_loss(logits, y)
      # metric
      preds = torch.argmax(logits, dim=1)
      acc = self.train_acc(preds, y)
      self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
      self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
      return loss

    # validation loop
    def validation_step(self, batch, batch_idx):
      x, y = batch
      logits = self(x)
      loss = F.nll_loss(logits, y)
      preds = torch.argmax(logits, dim=1)
      acc = self.val_acc(preds, y)
      self.log('val_loss', loss, prog_bar=True)
      self.log('val_acc', acc, prog_bar=True)
      return loss

    # test loop
    def test_step(self,batch,batch_idx):
      x,y=batch
      logits=self(x)
      loss=F.nll_loss(logits,y)

      pred=torch.argmax(logits,dim=1)
      acc=self.test_acc(pred,y)
      self.log('test_loss',loss,on_epoch=True)
      self.log('test_acc',acc,on_epoch=True)
      return loss

    def configure_optimizers(self):
      if self.optimizer == "sgd":
        optimizer = torch.optim.SGD(self.parameters(),self.learning_rate, momentum=0.9)
      elif self.optimizer == "adam":
        optimizer=torch.optim.Adam(self.parameters(),self.learning_rate)
      return optimizer



In [5]:
cifar = CifarDataModule(batch_size=32)
cifar.prepare_data()
cifar.setup()
# grab samples to log predictions on
samples = next(iter(cifar.val_dataloader()))

100%|██████████| 170498071/170498071 [00:03<00:00, 43901421.13it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [8]:
wandb.login(key='8793ca2cf88b7dcf86ef0ba0aabdbe2f48379870')
sweep_config = {
'method': 'random',
'name': 'sweep1',
'metric': {
    'goal': 'maximize',
    'name': 'val_acc'
    }
}
parameters_dict = {
    'optimizer': {'values': ['adam', 'sgd']},
    'learning_rate': {'values': [0.1, 0.01, 0.001, 0.05, 0.005]},
    'neurons_FC1': {'values': [256, 512, 128]},
    'neurons_FC2': {'values': [128, 256, 512, 64]},
    }

sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_acc'},
 'name': 'sweep1',
 'parameters': {'learning_rate': {'values': [0.1, 0.01, 0.001, 0.05, 0.005]},
                'neurons_FC1': {'values': [256, 512, 128]},
                'neurons_FC2': {'values': [128, 256, 512, 64]},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [12]:
def train_model(config=None):
    wandb.init(project='task4')
    config = wandb.config
    wandb_logger = WandbLogger(project="task4",job_type='train',log_model='all')

    # instantiate classes
    dm = CifarDataModule(32)
    dm.prepare_data()
    dm.setup()
    model = CIFAR10LitModel(config, (3, 32, 32), dm.num_classes)

    ### WandB
    wandb_logger.watch(model)

    trainer = pl.Trainer(max_epochs=20, logger=wandb_logger)
    # Train the model
    trainer.fit(model, dm)
    wandb.finish()



In [13]:
sweep_id=wandb.sweep(sweep_config, project="task4")
wandb.agent(sweep_id=sweep_id, function=train_model, count=5)

Create sweep with ID: ucbyg0qf
Sweep URL: https://wandb.ai/yan-meng/task4/sweeps/ucbyg0qf


wandb: Agent Starting Run: ucqkvxoo with config:
wandb: 	learning_rate: 0.1
wandb: 	neurons_FC1: 128
wandb: 	neurons_FC2: 512
wandb: 	optimizer: sgd
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_acc_epoch,▁▄▄▅▅▆▆▆▆▇▇▇▇▇██████
train_acc_step,▃▁▁▅▄▅▃▅▃▅▅▃▅▅▅▅▃▇▅▆▆▆▃▆▆█▄▅▇▄█▆▅▄▅▆▆▇█▇
train_loss_epoch,█▅▅▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁
train_loss_step,▇█▇▄▅▅▅▃▆▄▅▅▄▃▄▄▅▃▃▄▄▃▄▂▃▁▄▅▃▅▁▃▄▄▅▂▂▂▂▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_acc,▁▃▄▆▅▆▆▆▆▆▇▇▇▇█▇▇▇█▇
val_loss,█▅▄▃▃▂▂▂▃▂▂▁▁▂▁▂▂▂▂▃
epoch,19
train_acc_epoch,0.68856
train_acc_step,0.6875
